In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


In [ ]:
# %cd '/content/gdrive/MyDrive/Clara/'

/content/gdrive/MyDrive/Clara


# SKIP

In [ ]:
#mkdir BRAIN_DATA

In [ ]:
#cd BRAIN_DATA

/content/gdrive/My Drive/Clara/clara_new/BRAIN_DATA


In [ ]:
import os
import urllib.request
# urllib.request.urlretrieve('https://www.cbica.upenn.edu/sbia/Spyridon.Bakas/MICCAI_BraTS/2018/MICCAI_BraTS_2018_Data_Training.zip ', 'MICCAI_BraTS_2018_Data_Training.zip')


In [ ]:
# urllib.request.urlretrieve('https://www.cbica.upenn.edu/sbia/Spyridon.Bakas/MICCAI_BraTS/2018/MICCAI_BraTS_2018_Data_Training.zip ', 'MICCAI_BraTS_2018_Data_Training.zip')
# urllib.request.urlretrieve('https://www.cbica.upenn.edu/sbia/Spyridon.Bakas/MICCAI_BraTS/2018/MICCAI_BraTS_2018_Data_Validation.zip ', 'MICCAI_BraTS_2018_Data_Validation.zip')

In [ ]:
# !unzip MICCAI_BraTS_2018_Data_Training.zip

In [ ]:
# %cd '/content/gdrive/MyDrive/Clara/BRAIN_DATA/VAL/Brats18_CBICA_AAM_1'

/content/gdrive/MyDrive/Clara/BRAIN_DATA/VAL/Brats18_CBICA_AAM_1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models


In [ ]:
!python use_clara_pretrained_chestx_pytorch_model.py

torch.Size([1, 1, 256, 256])
torch.Size([1, 15])
tensor([[-3.9034, -6.3621, -9.4223, -7.4985, -3.5780, -7.8283, -7.4088, -7.3462,
         -8.9824, -6.4781, -3.4974, -4.7420, -4.3284, -5.7585, -5.1455]],
       grad_fn=<AddmmBackward>)


In [ ]:
# model = models.densenet121(pretrained=False)
# for child in model.children():
#     break
# model1 = child[:-1]

In [ ]:
#model1

In [ ]:
# optimizer = optim.SGD(densenet.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# # Specify a path
# PATH = "final_model.pt"

# # Save
# #torch.save(net.state_dict(), PATH)

# # Load
# #model = densenet()
# densenet.load_state_dict(torch.load(PATH))
# densenet.eval()

In [ ]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
from torch import Tensor
from typing import Any, List, Tuple
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models


In [ ]:
__all__ = ['DenseNet', 'densenet121', 'densenet169', 'densenet201', 'densenet161']


In [ ]:
class _DenseLayer(nn.Module):
    def __init__(
        self,
        num_input_features: int,
        growth_rate: int,
        bn_size: int,
        drop_rate: float,
        memory_efficient: bool = False
    ) -> None:
        super(_DenseLayer, self).__init__()
        self.norm1: nn.BatchNorm2d
        self.add_module('norm1', nn.BatchNorm2d(num_input_features))
        self.relu1: nn.ReLU
        self.add_module('relu1', nn.ReLU(inplace=True))
        self.conv1: nn.Conv2d
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False))
        self.norm2: nn.BatchNorm2d
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate))
        self.relu2: nn.ReLU
        self.add_module('relu2', nn.ReLU(inplace=True))
        self.conv2: nn.Conv2d
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False))
        self.drop_rate = float(drop_rate)
        self.memory_efficient = memory_efficient
    def bn_function(self, inputs: List[Tensor]) -> Tensor:
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))  # noqa: T484
        return bottleneck_output
    # todo: rewrite when torchscript supports any
    def any_requires_grad(self, input: List[Tensor]) -> bool:
        for tensor in input:
            if tensor.requires_grad:
                return True
        return False
    @torch.jit.unused  # noqa: T484
    def call_checkpoint_bottleneck(self, input: List[Tensor]) -> Tensor:
        def closure(*inputs):
            return self.bn_function(inputs)
        return cp.checkpoint(closure, *input)
    @torch.jit._overload_method  # noqa: F811
    def forward(self, input: List[Tensor]) -> Tensor:
        pass
    @torch.jit._overload_method  # noqa: F811
    def forward(self, input: Tensor) -> Tensor:
        pass
    # torchscript does not yet support *args, so we overload method
    # allowing it to take either a List[Tensor] or single Tensor
    def forward(self, input: Tensor) -> Tensor:  # noqa: F811
        if isinstance(input, Tensor):
            prev_features = [input]
        else:
            prev_features = input
        if self.memory_efficient and self.any_requires_grad(prev_features):
            if torch.jit.is_scripting():
                raise Exception("Memory Efficient not supported in JIT")
            bottleneck_output = self.call_checkpoint_bottleneck(prev_features)
        else:
            bottleneck_output = self.bn_function(prev_features)
        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return new_features

In [ ]:
class _DenseBlock(nn.ModuleDict):
    _version = 2
    def __init__(
        self,
        num_layers: int,
        num_input_features: int,
        bn_size: int,
        growth_rate: int,
        drop_rate: float,
        memory_efficient: bool = False
    ) -> None:
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module('denselayer%d' % (i + 1), layer)
    def forward(self, init_features: Tensor) -> Tensor:
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)

In [ ]:
class _Transition(nn.Sequential):
    def __init__(self, num_input_features: int, num_output_features: int) -> None:
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))

In [ ]:
class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_.
    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_.
    """
    def __init__(
        self,
        growth_rate: int = 32,
        block_config: Tuple[int, int, int, int] = (6, 12, 24, 16),
        num_init_features: int = 64,
        bn_size: int = 4,
        drop_rate: float = 0,
        num_classes: int = 15,
        memory_efficient: bool = False
    ) -> None:
        super(DenseNet, self).__init__()
        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(1, num_init_features, kernel_size=7, stride=2,
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),]))
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))
        # Linear layer
        self.class_layers = nn.Linear(num_features, num_classes)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)
    def forward(self, x: Tensor) -> Tensor:
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.class_layers(out)
        return out

In [ ]:
def _densenet(
    arch: str,
    growth_rate: int,
    block_config: Tuple[int, int, int, int],
    num_init_features: int,
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> DenseNet:
    model = DenseNet(growth_rate, block_config, num_init_features, **kwargs)
    return model

In [ ]:
def densenet121(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> DenseNet:
    r"""Densenet-121 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_.
    """
    return _densenet('densenet121', 32, (6, 12, 24, 16), 64, pretrained, progress,
                     **kwargs)

In [ ]:
# def densenet161(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> DenseNet:
#     r"""Densenet-161 model from
#     `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_.
#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#         progress (bool): If True, displays a progress bar of the download to stderr
#         memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
#           but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_.
#     """
#     return _densenet('densenet161', 48, (6, 12, 36, 24), 96, pretrained, progress,
#                      **kwargs)
# ​
# ​
# def densenet169(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> DenseNet:
#     r"""Densenet-169 model from
#     `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_.
#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#         progress (bool): If True, displays a progress bar of the download to stderr
#         memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
#           but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_.
#     """
#     return _densenet('densenet169', 32, (6, 12, 32, 32), 64, pretrained, progress,
#                      **kwargs)
# ​
# ​
# def densenet201(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> DenseNet:
#     r"""Densenet-201 model from
#     `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_.
#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#         progress (bool): If True, displays a progress bar of the download to stderr
#         memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
#           but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_.
#     """
#     return _densenet('densenet201', 32, (6, 12, 48, 32), 64, pretrained, progress,
#                      **kwargs)

In [ ]:
model=densenet121()

In [ ]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
ls

commands/  docs/           model.pt  model.ts    train_stats.json
config/    final_model.pt  models/   resources/


In [ ]:
  # Specify a path
PATH = "final_model.pt"
# Load
model=torch.load(PATH)
#model.eval()

In [ ]:
# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))
print(model)

In [ ]:
for i in model['model'].keys():
  print(i)

features.conv0.weight
features.norm0.weight
features.norm0.bias
features.norm0.running_mean
features.norm0.running_var
features.norm0.num_batches_tracked
features.denseblock1.denselayer1.layers.norm1.weight
features.denseblock1.denselayer1.layers.norm1.bias
features.denseblock1.denselayer1.layers.norm1.running_mean
features.denseblock1.denselayer1.layers.norm1.running_var
features.denseblock1.denselayer1.layers.norm1.num_batches_tracked
features.denseblock1.denselayer1.layers.conv1.weight
features.denseblock1.denselayer1.layers.norm2.weight
features.denseblock1.denselayer1.layers.norm2.bias
features.denseblock1.denselayer1.layers.norm2.running_mean
features.denseblock1.denselayer1.layers.norm2.running_var
features.denseblock1.denselayer1.layers.norm2.num_batches_tracked
features.denseblock1.denselayer1.layers.conv2.weight
features.denseblock1.denselayer2.layers.norm1.weight
features.denseblock1.denselayer2.layers.norm1.bias
features.denseblock1.denselayer2.layers.norm1.running_mean
fea

In [ ]:
ls

dn.py           model.ts          use_clara_pretrained_chestx_pytorch_model.py
final_model.pt  __pycache__/
model.pt        train_stats.json


# START FROM HERE

In [ ]:
%cd '/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models'

/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models


In [ ]:
# import os
# import urllib.request
# urllib.request.urlretrieve('https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz', 'image01.tar.gz')


In [ ]:
import torch
from dn import densenet121
from collections import OrderedDict
import cv2
import matplotlib.pyplot as plt
import re
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
from torch import Tensor
# from typing import Any, List, Tuple
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms
import numpy as np

In [ ]:
# cd '/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models'

In [ ]:
model = densenet121(pretrained=False)
wts = torch.load('final_model.pt',map_location=torch.device('cpu'))['model']
wts = OrderedDict((k.replace('.layers.','.') if '.layers.' in k else k, v) for k, v in wts.items())
wts = OrderedDict((k.replace('.out.','.') if '.out.' in k else k, v) for k, v in wts.items())
model.load_state_dict(wts)

<All keys matched successfully>

In [ ]:
op_list=['Nodule', 'Mass', 'Distortion of Pulmonary Architecture', 'Pleural Based Mass', 'Granuloma', 'Fluid in Pleural Space', 'Right Hilar Abnormality', 'Left Hilar Abnormality', 'Atelectasis', 'Infiltrate', 'Scarring', 'Pleural Fibrosis', 'Bone/Soft Tissue Lesion', 'Cardiac Abnormality', 'COPD']

In [ ]:
!pip3 install monai

     |████████████████████████████████| 501kB 16.1MB/s 


In [ ]:
import monai
obj = monai.transforms.NormalizeIntensity(subtrahend=2876.37, divisor= 883, nonzero=True, channel_wise=True)

In [ ]:
# image = obj._normalize(image)  # normalize

# SKIP

In [ ]:
# op_list=['Nodule', 'Mass', 'Distortion of Pulmonary Architecture', 'Pleural Based Mass', 'Granuloma', 'Fluid in Pleural Space', 'Right Hilar Abnormality', 'Left Hilar Abnormality', 'Atelectasis', 'Infiltrate', 'Scarring', 'Pleural Fibrosis', 'Bone/Soft Tissue Lesion', 'Cardiac Abnormality', 'COPD']

In [ ]:
import pandas as pd
lable_data=pd.read_csv('/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/Data_Entry_2017_v2020.csv')

In [ ]:
# lable_data['Finding Labels']=lable_data['Finding Labels'].str.split('|')
allclass=lable_data[lable_data['Finding Labels'].str.contains("Mass|Nodule|Atelectasis" )]

In [ ]:
allclass=allclass.reset_index().drop('index',axis=1)

In [ ]:
allclass=allclass[:200]

In [ ]:
allclass

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000004_000.png,Mass|Nodule,0,4,82,M,AP,2500,2048,0.168,0.168
1,00000008_002.png,Nodule,2,8,72,F,PA,2048,2500,0.168,0.168
2,00000011_006.png,Atelectasis,6,11,75,M,PA,2992,2991,0.143,0.143
3,00000012_000.png,Effusion|Mass,0,12,76,M,PA,2992,2991,0.143,0.143
4,00000013_022.png,Infiltration|Mass,0,13,56,M,PA,2870,2991,0.143,0.143
...,...,...,...,...,...,...,...,...,...,...,...
195,00000250_004.png,Mass,4,250,49,M,PA,2742,2761,0.143,0.143
196,00000250_014.png,Infiltration|Mass,14,250,52,M,PA,2992,2991,0.143,0.143
197,00000250_015.png,Mass,15,250,52,M,PA,2992,2885,0.143,0.143
198,00000250_016.png,Atelectasis|Infiltration|Mass,16,250,52,M,PA,2500,2048,0.168,0.168


In [ ]:
# SKIP BELOW

In [ ]:
# lblist=list(lable_data['Finding Labels'])

In [ ]:
import itertools 

In [ ]:
# lblist=list(itertools.chain.from_iterable(lblist))

In [ ]:
# nodup=set(lblist)

In [ ]:
op_list=['Nodule', 'Mass', 'Distortion of Pulmonary Architecture', 'Pleural Based Mass', 'Granuloma', 'Fluid in Pleural Space', 'Right Hilar Abnormality', 'Left Hilar Abnormality', 'Atelectasis', 'Infiltrate', 'Scarring', 'Pleural Fibrosis', 'Bone/Soft Tissue Lesion', 'Cardiac Abnormality', 'COPD']

In [ ]:
# final_class=[]
# for i in op_list:
#   if i in nodup:
#     final_class.append(i)

In [ ]:
# # lable_data['Finding Labels']=lable_data['Finding Labels'].astype('str')
# lable_data['Finding Labels']=lable_data['Finding Labels'].replace("[","",regex=True)
# lable_data['Finding Labels']=lable_data['Finding Labels'].replace("]","",regex=True)

In [ ]:
import os
import urllib.request


In [ ]:
urllib.request.urlretrieve('https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz', 'image01.tar.gz')


NameError: ignored

In [ ]:
%cd '/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA'

/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA


In [ ]:
!tar -xf '/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/image01.tar.gz'


In [ ]:
!find '/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA/images' -type f | wc -l

4999


In [ ]:
lable_data['Finding Labels']

0                    [Cardiomegaly]
1         [Cardiomegaly, Emphysema]
2          [Cardiomegaly, Effusion]
3                      [No Finding]
4                          [Hernia]
                    ...            
112115            [Mass, Pneumonia]
112116                 [No Finding]
112117                 [No Finding]
112118                 [No Finding]
112119                 [No Finding]
Name: Finding Labels, Length: 112120, dtype: object

In [ ]:
newdata=lable_data[lable_data['Finding Labels']]

In [ ]:
duplicate_df=lable_data.copy()

In [ ]:
duplicate_df['Finding Labels']

0                    [Cardiomegaly]
1         [Cardiomegaly, Emphysema]
2          [Cardiomegaly, Effusion]
3                      [No Finding]
4                          [Hernia]
                    ...            
112115            [Mass, Pneumonia]
112116                 [No Finding]
112117                 [No Finding]
112118                 [No Finding]
112119                 [No Finding]
Name: Finding Labels, Length: 112120, dtype: object

In [ ]:
duplicate_df=duplicate_df.reset_index().drop('index',axis=1)

In [ ]:
# for i in lable_data['Finding Label']:
#   if i in op_list:
#     continue
#   else:
#     lable_data.drop(lable_data[lable_data['Finding Label'] == i].index, inplace = True) 

In [ ]:
# lable_data=lable_data.reset_index().drop('index',axis=1)

In [ ]:
# lable_data['Finding Label'].value_counts()

Atelectasis    180
Infiltrate     123
Mass            85
Nodule          79
Name: Finding Label, dtype: int64

In [ ]:
# continue

In [ ]:
list_of_image=list(allclass['Image Index'])

In [ ]:
# lable_data.to_csv('4lables.csv',index=False)

In [ ]:
print('Before')

Before


In [ ]:
# !find '/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA/images' -type f | wc -l

4999


In [ ]:
folder_path = (r'/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA/images')
test = os.listdir(folder_path)

In [ ]:
allclass.head(1)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000004_000.png,Mass|Nodule,0,4,82,M,AP,2500,2048,0.168,0.168


In [ ]:
for images in test:
    if images not in list_of_image:
        os.remove(os.path.join(folder_path, images))

In [ ]:
print('after')
!find '/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA/images' -type f | wc -l

after
200


In [ ]:
# newdf=pd.read_csv('4lables.csv')

In [ ]:
# all_images = os.listdir(folder_path)

In [ ]:
allclass

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000004_000.png,Mass|Nodule,0,4,82,M,AP,2500,2048,0.168,0.168
1,00000008_002.png,Nodule,2,8,72,F,PA,2048,2500,0.168,0.168
2,00000011_006.png,Atelectasis,6,11,75,M,PA,2992,2991,0.143,0.143
3,00000012_000.png,Effusion|Mass,0,12,76,M,PA,2992,2991,0.143,0.143
4,00000013_022.png,Infiltration|Mass,0,13,56,M,PA,2870,2991,0.143,0.143
...,...,...,...,...,...,...,...,...,...,...,...
195,00000250_004.png,Mass,4,250,49,M,PA,2742,2761,0.143,0.143
196,00000250_014.png,Infiltration|Mass,14,250,52,M,PA,2992,2991,0.143,0.143
197,00000250_015.png,Mass,15,250,52,M,PA,2992,2885,0.143,0.143
198,00000250_016.png,Atelectasis|Infiltration|Mass,16,250,52,M,PA,2500,2048,0.168,0.168


In [ ]:
# for i in newdf['Image Index']:
#   if i in all_images:
#     continue
#   else:
#     newdf.drop(newdf[newdf['Image Index'] == i].index, inplace = True) 

In [ ]:
# newdf=newdf.reset_index().drop('index',axis=1)


In [ ]:
# CONTINUE THIS ONE

In [ ]:
newdf=allclass[['Image Index','Finding Labels']]

In [ ]:
# %cd '/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA/images'

/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA/images


# CONTINUE

In [ ]:
def predict(image):
  ii = cv2.imread(image)
  gray_image = cv2.cvtColor(ii, cv2.COLOR_BGR2GRAY)
  # print('gray_image_shape',gray_image.shape)
  resized_image = cv2.resize(gray_image, (256,256)) 
  resized_image = obj._normalize(resized_image)  # normalize
  # print('resized_image',resized_image.shape)
  trans = transforms.ToTensor()
  inp=trans(resized_image) # tensor
  # print('inp_shape',inp.shape) # tensor shape
  inp = inp.unsqueeze(0) # add pad ( 3d to 4d)
  # print('model(inp).shape',model(inp).shape)
  # plt.imshow(resized_image,cmap='gray')
  # plt.show()
  print('I/P Shape',inp.shape)
  img1=model(inp)
  print('img1',img1)
  img1=img1.detach().numpy()
  result=op_list[np.argmax(img1)]
  return result

In [ ]:
predict('/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA/images/00000004_000.png')

I/P Shape torch.Size([1, 1, 256, 256])
img1 tensor([[-3.8675, -6.3115, -9.3999, -7.4663, -3.5195, -7.8143, -7.3906, -7.2886,
         -8.9844, -6.5180, -3.5543, -4.7754, -4.2626, -5.8114, -5.1397]],
       grad_fn=<AddmmBackward>)


'Granuloma'

# SKIP

In [ ]:
newdf.head()

,Image Index,Finding Labels
0,00000004_000.png,Mass|Nodule
1,00000008_002.png,Nodule
2,00000011_006.png,Atelectasis
3,00000012_000.png,Effusion|Mass
4,00000013_022.png,Infiltration|Mass


In [ ]:
import tqdm
import PIL 
from PIL import Image 
  

In [ ]:
%cd '/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA'

/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA


In [ ]:
!pwd

/content/gdrive/MyDrive/Clara/CHESTXRAY/models/models/DATA


In [ ]:
# # importing cv2
# import cv2
# font = cv2.FONT_HERSHEY_SIMPLEX
# org = (50, 50)
# fontScale = 1
# color = (255, 0, 0)
# thickness = 2

In [ ]:
# !pip install ipython-autotime
# %load_ext autotime


time: 3.68 ms (started: 2021-04-27 07:52:41 +00:00)


In [ ]:
# for j,i in enumerate(zip(newdf['Image Index'],newdf['Finding Labels'])):
#   prediction=predict("images/" + str(i[0]))
#   im1 = Image.open("images/" + str(i[0])) 
#   im1.save( "results/" + str(j) + "_act_" + str(i[1]) + "_pred_" + str(prediction)  + ".png")
#   print('done')
#   break

[[-3.9053435 -6.289189  -9.344538  -7.474014  -3.4921386 -7.732866
  -7.327554  -7.2381387 -8.888096  -6.3841677 -3.5306008 -4.736046
  -4.363514  -5.73064   -5.097861 ]]
Granuloma
done
time: 6.76 s (started: 2021-04-27 07:52:51 +00:00)


In [ ]:
# importing cv2
# import cv2
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 0.8
color = (0, 255, 0)
thickness = 2

time: 2.03 ms (started: 2021-04-27 08:18:53 +00:00)


In [ ]:
# prediction=predict("images/" + '00000013_028.png')
# image = cv2.imread("images/" + '00000013_028.png')
# image = cv2.putText(image, "Actual:" + str(i[1]) + " Predicted:" + str(prediction), org, font, 
#                     fontScale, color, thickness, cv2.LINE_AA)

time: 5.75 s (started: 2021-04-27 08:17:19 +00:00)


In [ ]:
  # cv2.imwrite('Model Related/sdadad.png', image)

True

time: 61 ms (started: 2021-04-27 08:18:11 +00:00)


In [ ]:
from tqdm import tqdm

time: 745 µs (started: 2021-04-27 08:06:04 +00:00)


In [ ]:
from tqdm import tqdm
for i in tqdm(zip(newdf['Image Index'],newdf['Finding Labels'])):
  prediction=predict("images/" + str(i[0]))
  image = cv2.imread("images/" + str(i[0]))
  image = cv2.putText(image, "Actual:" + str(i[1]) + " Predicted:" + str(prediction), org, font, 
                      fontScale, color, thickness, cv2.LINE_AA)
  cv2.imwrite('Model Related/' + str(i[0]), image)



0it [00:00, ?it/s]

1it [00:05,  5.49s/it]

2it [00:10,  5.49s/it]

3it [00:16,  5.48s/it]

4it [00:21,  5.50s/it]

5it [00:27,  5.53s/it]

6it [00:33,  5.56s/it]

7it [00:38,  5.55s/it]

8it [00:44,  5.53s/it]

9it [00:49,  5.50s/it]

10it [00:55,  5.50s/it]

11it [01:00,  5.52s/it]

12it [01:06,  5.49s/it]

13it [01:11,  5.47s/it]

14it [01:17,  5.47s/it]

15it [01:22,  5.45s/it]

16it [01:27,  5.42s/it]

17it [01:33,  5.40s/it]

18it [01:38,  5.41s/it]

19it [01:44,  5.42s/it]

20it [01:49,  5.41s/it]

21it [01:54,  5.37s/it]

22it [02:00,  5.36s/it]

23it [02:05,  5.37s/it]

24it [02:10,  5.36s/it]

25it [02:16,  5.38s/it]

26it [02:21,  5.37s/it]

27it [02:26,  5.36s/it]

28it [02:32,  5.34s/it]

29it [02:37,  5.33s/it]

30it [02:42,  5.36s/it]

31it [02:48,  5.36s/it]

32it [02:54,  5.48s/it]

33it [03:00,  5.64s/it]

34it [03:06,  5.78s/it]

35it [03:12,  5.81s/it]

36it [03:17,  5.82s/it]

37it [03:23,  5.83s/it]

38it [03:30,  5.99s/it]

39it [03:36,  6.04s/it]

40it [03:42,

time: 19min 58s (started: 2021-04-27 08:19:09 +00:00)


In [ ]:
# value=[]
# for i in tqdm(newdf['Image Index']):
#   print(i)
#   value.append(predict(i))

In [ ]:
newdf=newdf.rename(columns={'Image Index':'Image','Finding Label':'Actual'})

In [ ]:
newdf['Predicted']=value

In [ ]:
newdf.tail(10)

,Image Index,Finding Labels
190,00000248_022.png,Effusion|Infiltration|Mass
191,00000248_023.png,Mass
192,00000248_024.png,Mass
193,00000250_001.png,Atelectasis|Nodule
194,00000250_003.png,Nodule
195,00000250_004.png,Mass
196,00000250_014.png,Infiltration|Mass
197,00000250_015.png,Mass
198,00000250_016.png,Atelectasis|Infiltration|Mass
199,00000251_001.png,Atelectasis


In [ ]:
inp = torch.randn(1,1,256,256)
print(inp.shape)
print(model(inp).shape)
print(model(inp))